# Implementing Various Parts of HMM

- Two hidden states : [1_Hot, 2_Cold]
- Three observed states : [0_Small, 1_Medium, 2_Large]
- Init Probability : $\pi = [0.6, 0.4]$
- Transition Matrix $A$

| t-1 => t | s0 | s_hot | s_cold |
|----------|----|-------|--------|
| **s0**       | 0  | 0.6   | 0.4    |
| **s_hot**    | 0  | 0.7   | 0.3    |
| **s_cold**   | 0  | 0.4   | 0.6    |


- Emission Matrix $B$

| zt => xt | 0_small | 1_medium | 2_large |
|----------|---------|----------|---------|
| s0       | NA       | NA        | NA       |
| s_hot    | 0.1     | 0.4      | 0.5     |
| s_cold   | 0.7     | 0.2      | 0.1     |

In [2]:
## In numpy
import numpy as np

In [3]:
vocStates = 2
vocObs = 3

trA = np.array(
    [
        [0, 0.6, 0.4],
        [0, 0.7, 0.3],
        [0, 0.4, 0.6]
    ]
)

emB = np.array(
    [
        [None, None, None],
        [0.1, 0.4, 0.5],
        [0.7, 0.2, 0.1]
    ]
)

obX = [0, 1, 0, 2]

### Task 1
Given $A, B, \pi$ and a observed sequence $\vec{x} = (0, 1, 0, 2)$, what is the probability of observing this sequence?

As derived, for a sequence $\vec x$ given $A, B$, we can arrive at the following:
$$
P(\vec x; A,B) = \sum_{\vec z} P(\vec x, \vec z; A,B) \\
= \sum_{\vec z} \big(\prod_{t=1}^T B_{z_t, x_t}\big) \big(\prod_{t=1}^T A_{z_{t-1}, z_t}\big) 
$$

This solution involves considering every possible state assignment combination for the output sequence.

In [4]:
total_prob = 0
for i in range(1, 3):
    for j in range(1, 3):
        for k in range(1, 3):
            for l in range(1, 3):
                zprob = 1.0
                zseq = (i, j, k, l)
                for t, (z, x) in enumerate(zip(zseq, obX)):
                    _b = emB[z, x]
                    
                    if t == 0:
                        zprev = 0
                    else:
                        zprev = zseq[t-1]
                    
                    _a = trA[zprev, z]
                    
                    zprob = zprob * _a * _b
                    
                total_prob += zprob
                print zseq, round(zprob, 6), total_prob
                
print total_prob

(1, 1, 1, 1) 0.000412 0.0004116
(1, 1, 1, 2) 3.5e-05 0.00044688
(1, 1, 2, 1) 0.000706 0.00115248
(1, 1, 2, 2) 0.000212 0.00136416
(1, 2, 1, 1) 5e-05 0.0014145599999999998
(1, 2, 1, 2) 4e-06 0.0014188799999999998
(1, 2, 2, 1) 0.000302 0.0017212799999999997
(1, 2, 2, 2) 9.1e-05 0.0018119999999999998
(2, 1, 1, 1) 0.001098 0.0029095999999999996
(2, 1, 1, 2) 9.4e-05 0.0030036799999999995
(2, 1, 2, 1) 0.001882 0.004885279999999999
(2, 1, 2, 2) 0.000564 0.005449759999999999
(2, 2, 1, 1) 0.00047 0.005920159999999999
(2, 2, 1, 2) 4e-05 0.005960479999999999
(2, 2, 2, 1) 0.002822 0.008782879999999998
(2, 2, 2, 2) 0.000847 0.009629599999999999
0.009629599999999999


This solution is naive. We are iterating over every combination of $\vec z$ over time. Let's see if it can be done faster.

The task is to find $P(\vec x)$ (matrices $A$, $B$ omitted for brevity). This can be expressed as
$$
P(\vec x) = \sum_{z_t} P(z_t, \vec x)  \tag0
$$

The equation above is true for any expression. Let's now try to expand $P(z_t, \vec x)$ in terms of our emission and transition terms ...

$$
\begin{align}
P(z_t, x_{1:t}) &= \sum_{z_{t-1}} P(z_t, z_{t-1}, x_{1:t}) \tag1 \\
&= \sum_{z_{t-1}} P(x_t, z_t, z_{t-1}, x_{t-1}, x_{t-2}, ... , x_1) \tag2 \\
&= \sum_{z_{t-1}} P(x_t | z_t) P(z_t, z_{t-1}, x_{1:t-1}) \tag3 \\
&= \sum_{z_{t-1}} P(x_t | z_t) P(z_t | z_{t-1}) P(z_{t-1}, x_{1:t-1}) \tag4 \\
&= P(x_t | z_t) \sum_{z_{t-1}} P(z_t | z_{t-1}) P(z_{t-1}, x_{1:t-1}) \tag5
\end{align}
$$

In eq 1, we introduce a new hidden state var from previous timestep. Eq 2 simple expands $x_{1:t}$ before applying the Markov assumptions. In eq 3 we apply the emission prob. assumption and in eq 4 we apply the transition prob. assumption. Finally, in eq 5, we can take $P(x_t, z_t)$ out of the summand since we are iterating over values of $z_{t-1}$ only.

Observe that there is a recurrence b/w EQ1 and EQ5. If we define $\alpha_z(t)$ to be $P(z_t, x_{1:t})$, then we can rewrite EQ 5 as follows:

$$
\begin{align}
P(z_t, x_{1:t}) &= P(x_t | z_t) \sum_{z_{t-1}} P(z_t | z_{t-1}) P(z_{t-1}, x_{1:t-1}) \\
\rightarrow \alpha_z(t) &= P(x_t | z_t) \sum_{z_{t-1}} P(z_t | z_{t-1}) \alpha_z(t-1) \\
&= B_{z_t, x_t} \sum_{z_{t-1}} A_{z_t, z_{t-1}} \alpha_z(t-1)
\end{align}
$$

This expression only involves summing over all possible states of $z_i$, which in our example is 2. Our original expression EQ 0, thus becomes

$$
P(\vec x) = \sum_{z_t} P(z_t, \vec x) = \sum_z \alpha_z(t = T)
$$

Now, we are no longer considering all possible combinations of the entire sequence of hidden states, but rather, the possible values of $z$, which in our example is 2. And to calculate $\alpha(T)$, we will have to iterate over all timesteps.

In [5]:
from copy import deepcopy
from IPython.core.debugger import set_trace

In [6]:
_toy = """
====
Init
====
_|_H_|_L_
*|0.5|0.5

============
Transmission
============
_|_H_|_L_
H|0.5|0.5
L|0.4|0.6

========
Emission
========
_|_A_|_C_|_G_|_T_
H|0.2|0.3|0.3|0.2
L|0.3|0.2|0.2|0.3

========
Observed
========
[G G C A]
"""
states = ['H', 'L']
genes = ['A', 'C', 'G', 'T']

inits = [0.5, 0.5]

trans = np.array(
    [
        [0.5, 0.5],
        [0.4, 0.6]
    ]
)
emmit = np.array(
    [
        [0.2, 0.3, 0.3, 0.2],
        [0.3, 0.2, 0.2, 0.3]
    ]
)

obsX = [2, 2, 1, 0]

In [7]:
## Priyam, forget whatever shit you wrote on top.
## We define `\alpha` for every timestep AND every hidden state ==> `\alpha_i(t)`.
total_prob = 0

for i in range(2):
    for j in range(2):
        for k in range(2):
            for l in range(2):
                prob = 1.0
                zstates = (i, j, k, l)
                
                for t, (z, x) in enumerate(zip(zstates, obsX)):
                    if t == 0:
                        prob = prob * inits[z] * emmit[z, x]
                    else:
                        prob = prob * trans[zstates[t-1], z] * emmit[z, x]
                
                total_prob += prob
                print zstates, prob, total_prob

(0, 0, 0, 0) 0.0003375 0.0003375
(0, 0, 0, 1) 0.00050625 0.00084375
(0, 0, 1, 0) 0.00018 0.00102375
(0, 0, 1, 1) 0.0004049999999999999 0.00142875
(0, 1, 0, 0) 0.00018 0.00160875
(0, 1, 0, 1) 0.00027 0.00187875
(0, 1, 1, 0) 0.000144 0.00202275
(0, 1, 1, 1) 0.000324 0.00234675
(1, 0, 0, 0) 0.00018000000000000004 0.00252675
(1, 0, 0, 1) 0.00027 0.0027967499999999998
(1, 0, 1, 0) 9.600000000000004e-05 0.00289275
(1, 0, 1, 1) 0.00021600000000000005 0.00310875
(1, 1, 0, 0) 0.000144 0.00325275
(1, 1, 0, 1) 0.00021600000000000002 0.00346875
(1, 1, 1, 0) 0.00011520000000000001 0.0035839500000000002
(1, 1, 1, 1) 0.0002592 0.00384315


In [8]:
print _toy


====
Init
====
_|_H_|_L_
*|0.5|0.5

Transmission
_|_H_|_L_
H|0.5|0.5
L|0.4|0.6

Emission
_|_A_|_C_|_G_|_T_
H|0.2|0.3|0.3|0.2
L|0.3|0.2|0.2|0.3

Observed
[G G C A]



In [10]:
for t,x in enumerate(obsX):
    if t == 0:
        alpha_prev = []
        for i in range(2):
            alpha_prev.append(inits[i] * emmit[i, x])
            
    else:
        alpha_new = []
        for i in range(2):
            _jsum = 0
            for j in range(2):
                _jsum += alpha_prev[j] * trans[j, i]
            
            alpha_new.append(_jsum * emmit[i, x])
        
        alpha_prev = deepcopy(alpha_new)
        
print sum(alpha_new)
assert round(total_prob, 6) == round(sum(alpha_new), 6)
print "Matches brute force!"

0.0038431500000000005
Matches brute force!


Now that we have it working, let's try deriving the *alpha-pass* algorithm again.

Following EQ 5, we can express the joint probability $P(z_t, x_{1:t})$ as 

$$
\begin{align}
P(z_t, x_{1:t}) &= P(x_t | z_t) \sum_{z_{t-1}} P(z_t | z_{t-1}) P(z_{t-1}, x_{1:t-1}) \tag6
\end{align}
$$

Here, $z_t$ is simply a random variable whose value exists in the set of state values $S$; the joint distribution "table" is defined for all possible values that $z_t$ can have. Concretely, we can define the joint probability for a particular state value $s_i$ as:

$$
\begin{align}
P(z_t=s_i, x_{1:t}) &= P(x_t | z_t=s_i) \sum_{s_j \in S} P(z_t = s_i | z_{t-1} = s_j) \times P(z_{t-1} = s_j, x_{1:t-1}) \tag7
\end{align}
$$

With this view, we can define $\alpha_t(s_i)$, as the probability of the observed partial sequnce upto time $t$ where $z_t = s_i$ :

$$
\alpha_t(s_i) = P(z_t = s_i, x_{1:t}) \tag8
$$

Now, we can re-write EQ7 in terms of $\alpha, A, B$:

$$
\alpha_t(s_i) = B_{s_i, x_t} \sum_{s_j \in S} A_{s_j, s_i} \times \alpha_{t-1}(s_j) \tag9
$$

The base-case, for this recursive definition is for the first observation $x_1$. This is simply the probability of observing $x_1$ for all possible state assignment to $z_1$.

$$
\alpha_1(s_i) = B_{s_i, x_1} \pi_{s_i} \tag{10}
$$

Using EQs 8, 9 and 10, we can express the probability of observing any sequnce $x_{1:T}$ as 

$$
\begin{align}
P(x_{1:T}) &= \sum_{s_i \in S} P(x_{1:T}, z_T=s_i) \\
&= \sum_{s_i \in S} \alpha_T(s_i)
\end{align}
$$

Finally! We have now reduced the time-complexity from $O(|S|^T)$ to $O(|S|^2T)$. How?
- To arrive at $\alpha_T$, we will have to loop over all possible values of $T$.
```python
for t,x in enumerate(obsX):
```

- At every iteration, we will compute the intermediate value $\alpha_t(s_i)$ for all $i$, which will require a sum over every state value $j$ from the previously computer $\alpha_{t-1}$ (from EQ 9) -- hence the $|S|^2$.
```python
        alpha_new = []
        for i in range(2): ## For current $\alpha_t$
            _jsum = 0
            for j in range(2): ## From previous $\alpha_{t-1}$
                _jsum += alpha_prev[j] * trans[j, i]
            
            alpha_new.append(_jsum * emmit[i, x])
        
        alpha_prev = deepcopy(alpha_new)
```

Once the last `alpha_new` has been computed, the joint is simply the sum of probabilities over the states : `return sum(alpha_new)`